In [1]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *
from src.validation import validate_jumpstart_cube, display_validate_results
from src.coherence import analyze_deck_theme_coherence_enhanced
from src.improve import apply_swap

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
cube_df = pd.read_csv('JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv')

# Optimise

In [17]:
# You can also analyze a specific deck by name
# Example: Analyze the "Green Big Creatures" deck

import importlib
import src.process
importlib.reload(src.process)

from src.process import optimize_deck_coherence, clear_swap_history

# Clear any previous swap history to prevent issues with oscillation
clear_swap_history()

coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
total_coherence = sum(result['overall_coherence'] for result in coherence.values())

improvement = True
iteration = 0
max_iterations = 20  # Prevent infinite loops

while improvement and iteration < max_iterations:
    prev_total_coherence = total_coherence
    cube_df = optimize_deck_coherence(cube_df=cube_df, oracle_df=oracle_df)
    coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
    total_coherence = sum(result['overall_coherence'] for result in coherence.values())
    improvement = total_coherence > prev_total_coherence
    iteration += 1
    print(f"Iteration {iteration}: Total coherence: {total_coherence:.2f} (improvement: {total_coherence - prev_total_coherence:.2f})")
    
    if not improvement:
        print("No further improvements found - optimization complete!")
    elif iteration >= max_iterations:
        print("Maximum iterations reached - stopping optimization")

Swap history cleared


Analyzing deck: Blue Card Draw

Current coherence: 2.0

Expected themes: Card Draw

Deck colors: U

Found 39 candidate cards to consider

Identified 13 cards as potential removal candidates

# 🔄 Swap Recommendations for Blue Card Draw

❌ **No beneficial swaps found.** The deck may already be well-optimized, or there may not be suitable replacement cards available.

No available swaps to apply for this deck.
Card: Behold the Multiverse, Score: 4, Themes: ['Card Draw']
Card: Lórien Revealed, Score: 4, Themes: ['Card Draw']
Card: Moon-Circuit Hacker, Score: 3, Themes: ['Card Draw']
Card: Opt, Score: 4, Themes: ['Card Draw']
Card: Young Blue Dragon, Score: 3, Themes: ['Card Draw']
Card: Brainstorm, Score: 4, Themes: ['Card Draw']
Card: Condescend, Score: 3, Themes: ['Card Draw']
Card: Preordain, Score: 4, Themes: ['Card Draw']
Card: Repeal, Score: 3, Themes: ['Card Draw']
Card: Into the Roil, Score: 3, Themes: ['Card Draw']
Card: Serum Visionary, Score: 3, Themes: ['Card Draw']
Card: Faerie Seer, Score: 3, Themes: ['Card Draw']
Card: Candy Trail, Score: 3, Themes: ['Card Draw']
1.9830769230769232
{'expected_themes': ['Card Draw'], 'theme_score': 3.3846153846153846, 'theme_matches': [{'card': 'Behold the Multiverse', 'score': 4, 'themes': ['Card Draw']}, {'card': 'Lórien Revealed', 'score': 4, 'themes': ['Card Draw']}, {'card': 'Moon-Circuit Hacker', 

# Save to file

In [3]:
# from src.export import export_cube_to_csv

# export_cube_to_csv(cube_df, oracle_df, 'JumpstartCube_ThePauperCube_ULTIMATE_Final_v2.csv')

# Analysis

In [4]:
# from src.coherence import display_coherence_analysis_enhanced

# display_coherence_analysis_enhanced(analyze_deck_theme_coherence_enhanced(cube_df, oracle_df))

In [6]:
# Let's first check the structure of the cube_df
print("Column names in cube_df:")
print(cube_df.columns.tolist())
print("\nFirst few rows:")
print(cube_df.head())

Column names in cube_df:
['Name', 'Set', 'Collector Number', 'Rarity', 'Color Identity', 'Type', 'Mana Cost', 'CMC', 'Power', 'Toughness', 'Tags']

First few rows:
                    Name    Set  Collector Number  Rarity Color Identity  \
0          Archaeomancer  Mixed               NaN  common              U   
1  Behold the Multiverse  Mixed               NaN  common              U   
2             Ephemerate  Mixed               NaN  common              W   
3         Lyev Skyknight  Mixed               NaN  common             WU   
4        Lórien Revealed  Mixed               NaN  common              U   

                      Type  Mana Cost  CMC  Power  Toughness  \
0  Creature - Human Wizard        NaN    4    NaN        NaN   
1                  Instant        NaN    4    NaN        NaN   
2                  Instant        NaN    1    NaN        NaN   
3  Creature - Human Knight        NaN    3    NaN        NaN   
4                  Sorcery        NaN    5    NaN        Na

In [7]:
# Let's examine the "Blue Card Draw" deck specifically
blue_card_draw_deck = cube_df[cube_df['Tags'] == 'Blue Card Draw']
print("Current Blue Card Draw deck:")
print("=" * 50)
for _, card in blue_card_draw_deck.iterrows():
    print(f"{card['Name']} - {card['Mana Cost']} - {card['Type']}")

print(f"\nTotal cards in deck: {len(blue_card_draw_deck)}")
print(f"Expected total: {TOTAL_CARDS}")

# Also let's check if we have the oracle data for these cards
print("\n" + "=" * 50)
print("Checking oracle data for these cards:")
for _, card in blue_card_draw_deck.iterrows():
    oracle_card = oracle_df[oracle_df['name'] == card['Name']]
    if not oracle_card.empty:
        oracle_text = oracle_card.iloc[0]['oracle_text'] if 'oracle_text' in oracle_card.columns else 'No oracle text'
        print(f"\n{card['Name']}:")
        print(f"  Oracle text: {oracle_text}")
    else:
        print(f"\n{card['Name']}: NOT FOUND in oracle_df")

Current Blue Card Draw deck:
Behold the Multiverse - nan - Instant
Lórien Revealed - nan - Sorcery
Moon-Circuit Hacker - nan - Enchantment Creature - Human Ninja
Opt - nan - Instant
Young Blue Dragon - nan - Creature - Dragon
Brainstorm - nan - Instant
Condescend - nan - Instant
Preordain - nan - Sorcery
Repeal - nan - Instant
Into the Roil - nan - Instant
Serum Visionary - nan - Creature - Vedalken Wizard
Faerie Seer - nan - Creature - Faerie Wizard
Candy Trail - nan - Artifact - Food Clue

Total cards in deck: 13
Expected total: 13

Checking oracle data for these cards:

Behold the Multiverse:
  Oracle text: No oracle text

Lórien Revealed:
  Oracle text: No oracle text

Moon-Circuit Hacker:
  Oracle text: No oracle text

Opt:
  Oracle text: No oracle text

Young Blue Dragon:
  Oracle text: No oracle text

Brainstorm:
  Oracle text: No oracle text

Condescend:
  Oracle text: No oracle text

Preordain:
  Oracle text: No oracle text

Repeal:
  Oracle text: No oracle text

Into the Roil

In [8]:
# Check oracle_df structure
print("Oracle_df columns:")
print(oracle_df.columns.tolist())
print("\nSample oracle_df row:")
print(oracle_df.head(1))

# Now let's get proper oracle data for Blue Card Draw cards
print("\n" + "=" * 50)
print("Blue Card Draw deck with oracle data:")
for _, card in blue_card_draw_deck.iterrows():
    oracle_card = oracle_df[oracle_df['name'] == card['Name']]
    if not oracle_card.empty:
        oracle_row = oracle_card.iloc[0]
        # Get the text column (might be 'text' instead of 'oracle_text')
        text_col = 'text' if 'text' in oracle_card.columns else 'oracle_text' if 'oracle_text' in oracle_card.columns else None
        oracle_text = oracle_row[text_col] if text_col else 'No text found'
        mana_cost = oracle_row['mana_cost'] if 'mana_cost' in oracle_card.columns else 'Unknown'
        type_line = oracle_row['type_line'] if 'type_line' in oracle_card.columns else oracle_row['type'] if 'type' in oracle_card.columns else 'Unknown'
        
        print(f"\n{card['Name']} ({mana_cost}) - {type_line}")
        print(f"  Text: {oracle_text}")
    else:
        print(f"\n{card['Name']}: NOT FOUND in oracle_df")

Oracle_df columns:
['name', 'CMC', 'Type', 'Color', 'Color Category', 'Oracle Text', 'tags', 'MTGO ID', 'Power', 'Toughness']

Sample oracle_df row:
          name  CMC                      Type Color Color Category  \
0  Boros Elite    1  Creature - Human Soldier     W          White   

                                         Oracle Text tags   MTGO ID  Power  \
0  Battalion — Whenever this creature and at leas...  NaN  120547.0    1.0   

   Toughness  
0        1.0  

Blue Card Draw deck with oracle data:

Behold the Multiverse (Unknown) - Unknown
  Text: No text found

Lórien Revealed (Unknown) - Unknown
  Text: No text found

Moon-Circuit Hacker (Unknown) - Unknown
  Text: No text found

Opt (Unknown) - Unknown
  Text: No text found

Young Blue Dragon (Unknown) - Unknown
  Text: No text found

Brainstorm (Unknown) - Unknown
  Text: No text found

Condescend (Unknown) - Unknown
  Text: No text found

Preordain (Unknown) - Unknown
  Text: No text found

Repeal (Unknown) - Unknown


In [9]:
# Let's try to find these cards with partial matching
print("Searching for Blue Card Draw cards in oracle_df:")
print("=" * 50)

# Get unique card names from the Blue Card Draw deck
blue_cards = blue_card_draw_deck['Name'].tolist()

for card_name in blue_cards:
    # Try exact match first
    exact_match = oracle_df[oracle_df['name'] == card_name]
    if not exact_match.empty:
        oracle_row = exact_match.iloc[0]
        oracle_text = oracle_row['Oracle Text'] if pd.notna(oracle_row['Oracle Text']) else 'No text'
        print(f"\n✓ {card_name} (EXACT MATCH)")
        print(f"  Type: {oracle_row['Type']}")
        print(f"  Text: {oracle_text}")
    else:
        # Try partial match
        partial_matches = oracle_df[oracle_df['name'].str.contains(card_name.split()[0], case=False, na=False)]
        if not partial_matches.empty:
            print(f"\n~ {card_name} (PARTIAL MATCHES):")
            for _, match in partial_matches.head(3).iterrows():  # Show top 3 matches
                match_text = match['Oracle Text'] if pd.notna(match['Oracle Text']) else 'No text'
                print(f"    - {match['name']}: {match_text[:80]}...")
        else:
            print(f"\n✗ {card_name} (NOT FOUND)")

# Let's also see what blue card draw spells are available in the oracle
print(f"\n\n{'='*60}")
print("Available blue cards that mention 'draw' in oracle_df:")
print("=" * 60)

blue_draw_cards = oracle_df[
    (oracle_df['Color Category'].str.contains('Blue', na=False)) & 
    (oracle_df['Oracle Text'].str.contains('draw', case=False, na=False))
]

print(f"Found {len(blue_draw_cards)} blue cards that mention 'draw'")
for _, card in blue_draw_cards.head(20).iterrows():  # Show first 20
    oracle_text = card['Oracle Text'] if pd.notna(card['Oracle Text']) else 'No text'
    print(f"\n{card['name']} ({card['CMC']} CMC) - {card['Type']}")
    print(f"  Text: {oracle_text}")

Searching for Blue Card Draw cards in oracle_df:

✓ Behold the Multiverse (EXACT MATCH)
  Type: Instant
  Text: Scry 2, then draw two cards. | Foretell {1}{U} (During your turn, you may pay {2} and exile this card from your hand face down. Cast it on a later turn for its foretell cost.)

✓ Lórien Revealed (EXACT MATCH)
  Type: Sorcery
  Text: Draw three cards. | Islandcycling {1} ({1}, Discard this card: Search your library for an Island card, reveal it, put it into your hand, then shuffle.)

✓ Moon-Circuit Hacker (EXACT MATCH)
  Type: Enchantment Creature - Human Ninja
  Text: Ninjutsu {U} ({U}, Return an unblocked attacker you control to hand: Put this card onto the battlefield from your hand tapped and attacking.) | Whenever this creature deals combat damage to a player, you may draw a card. If you do, discard a card unless this creature entered this turn.

✓ Opt (EXACT MATCH)
  Type: Instant
  Text: Scry 1. (Look at the top card of your library. You may put that card on the bottom.

In [10]:
# Let's analyze the Blue Card Draw theme more systematically

# First, let's see what the theme definition looks like in consts.py
print("THEME ANALYSIS: Blue Card Draw")
print("=" * 50)
print("Theme keywords from consts.py:", theme_keywords.get('Card Draw', []))
print("Color synergies for Blue:", color_synergies.get('U', []))

# Now let's look for quintessential blue card draw spells
print(f"\n{'='*50}")
print("QUINTESSENTIAL BLUE CARD DRAW SPELLS:")
print("=" * 50)

# Search for classic card draw spells
classic_draw_spells = [
    'Opt', 'Preordain', 'Brainstorm', 'Ponder', 'Divination', 
    'Think Twice', 'Accumulated Knowledge', 'Deep Analysis',
    'Fact or Fiction', 'Compulsive Research', 'Catalog',
    'Serum Visions', 'Consider', 'Impulse'
]

print("Looking for classic card draw spells in oracle_df:")
found_classics = []
for spell in classic_draw_spells:
    matches = oracle_df[oracle_df['name'].str.contains(spell, case=False, na=False)]
    if not matches.empty:
        for _, match in matches.iterrows():
            if spell.lower() in match['name'].lower():
                found_classics.append(match)
                oracle_text = match['Oracle Text'] if pd.notna(match['Oracle Text']) else 'No text'
                print(f"\n✓ {match['name']} ({match['CMC']} CMC)")
                print(f"  Text: {oracle_text}")

print(f"\n{'='*50}")
print("CURRENT BLUE CARD DRAW DECK ANALYSIS:")
print("=" * 50)

# Analyze current deck composition
current_deck = ['Behold the Multiverse', 'Lórien Revealed', 'Moon-Circuit Hacker', 
                'Opt', 'Young Blue Dragon', 'Brainstorm', 'Condescend', 
                'Preordain', 'Repeal', 'Into the Roil', 'Serum Visionary', 
                'Faerie Seer', 'Candy Trail']

print("Current deck composition:")
for card in current_deck:
    print(f"- {card}")

print(f"\nDeck categorization:")
print("Classic card draw: Opt, Brainstorm, Preordain")  
print("Bounce spells: Repeal, Into the Roil")
print("Creatures: Moon-Circuit Hacker, Young Blue Dragon, Serum Visionary, Faerie Seer")
print("Modal/Kicker: Behold the Multiverse, Lórien Revealed")
print("Counter/Utility: Condescend")
print("Artifact: Candy Trail")

THEME ANALYSIS: Blue Card Draw
Theme keywords from consts.py: ['draw', 'card', 'hand', 'library', 'cantrip', 'scry']
Color synergies for Blue: ['draw', 'counter', 'flying', 'scry', 'bounce', 'wizard', 'merfolk']

QUINTESSENTIAL BLUE CARD DRAW SPELLS:
Looking for classic card draw spells in oracle_df:

✓ Opt (1 CMC)
  Text: Scry 1. (Look at the top card of your library. You may put that card on the bottom.) | Draw a card.

✓ Preordain (1 CMC)
  Text: Scry 2, then draw a card. (To scry 2, look at the top two cards of your library, then put any number of them on the bottom and the rest on top in any order.)

✓ Brainstorm (1 CMC)
  Text: Draw three cards, then put two cards from your hand on top of your library in any order.

✓ Ponder (1 CMC)
  Text: No text

✓ Deep Analysis (4 CMC)
  Text: Target player draws two cards. | Flashback—{1}{U}, Pay 3 life. (You may cast this card from your graveyard for its flashback cost. Then exile it.)

✓ Compulsive Research (3 CMC)
  Text: Target player dr

In [11]:
# Let's analyze the coherence of the current Blue Card Draw deck
print("COHERENCE ANALYSIS:")
print("=" * 50)

# Check what we can find about the current deck's coherence
blue_coherence = coherence.get('Blue Card Draw', {})
if blue_coherence:
    print(f"Current Blue Card Draw coherence score: {blue_coherence.get('overall_coherence', 'N/A')}")
    print(f"Theme alignment: {blue_coherence.get('theme_alignment', 'N/A')}")
    print(f"Synergy score: {blue_coherence.get('synergy_score', 'N/A')}")
else:
    print("No coherence data found for Blue Card Draw deck")

print(f"\n{'='*50}")
print("RECOMMENDATIONS FOR IMPROVEMENT:")
print("=" * 50)

print("ISSUES WITH CURRENT DECK:")
print("1. Bounce spells (Repeal, Into the Roil) don't directly draw cards")
print("2. Young Blue Dragon is a big creature, not card draw focused") 
print("3. Candy Trail is an artifact that doesn't fit the pure card draw theme")
print("4. Mix of themes dilutes the card draw focus")

print("\nBETTER ALTERNATIVES FOUND IN ORACLE:")
print("✓ Serum Visions - Pure card draw with scry")
print("✓ Consider - Modern card selection with surveil")
print("✓ Deep Analysis - Card advantage with flashback")
print("✓ Compulsive Research - Raw card draw")
print("✓ Impulse - Card selection")

print("\nQUINTESSENTIAL BLUE CARD DRAW THEME SHOULD INCLUDE:")
print("- Low-cost cantrips (Opt, Preordain, Brainstorm)")
print("- Card selection (Ponder, Serum Visions, Impulse)")
print("- Card advantage spells (Divination effects)")
print("- Creatures that draw cards when they enter")
print("- Instant-speed card draw for flexibility")

print("\nTHEME DEFINITION ANALYSIS:")
print("Current 'Card Draw' keywords:", theme_keywords['Card Draw'])
print("✓ Good: covers 'draw', 'card', 'scry'")  
print("? Missing: 'surveil', 'look', 'search library', 'impulse draw'")

print("\nSUGGESTED THEME IMPROVEMENTS:")
print("Add keywords: 'surveil', 'look at', 'reveal', 'impulse', 'exile.*play'")
print("Consider separating 'Card Selection' vs 'Card Advantage' sub-themes")

COHERENCE ANALYSIS:
Current Blue Card Draw coherence score: 1.9830769230769232
Theme alignment: N/A
Synergy score: N/A

RECOMMENDATIONS FOR IMPROVEMENT:
ISSUES WITH CURRENT DECK:
1. Bounce spells (Repeal, Into the Roil) don't directly draw cards
2. Young Blue Dragon is a big creature, not card draw focused
3. Candy Trail is an artifact that doesn't fit the pure card draw theme
4. Mix of themes dilutes the card draw focus

BETTER ALTERNATIVES FOUND IN ORACLE:
✓ Serum Visions - Pure card draw with scry
✓ Consider - Modern card selection with surveil
✓ Deep Analysis - Card advantage with flashback
✓ Compulsive Research - Raw card draw
✓ Impulse - Card selection

QUINTESSENTIAL BLUE CARD DRAW THEME SHOULD INCLUDE:
- Low-cost cantrips (Opt, Preordain, Brainstorm)
- Card selection (Ponder, Serum Visions, Impulse)
- Card advantage spells (Divination effects)
- Creatures that draw cards when they enter
- Instant-speed card draw for flexibility

THEME DEFINITION ANALYSIS:
Current 'Card Draw' ke

In [12]:
# Let's find specific better alternatives from the oracle_df
print("SEARCHING FOR BETTER ALTERNATIVES:")
print("=" * 50)

# Search for blue cards that are more focused on card draw
better_alternatives = []

# Look for creatures that draw cards
card_draw_creatures = oracle_df[
    (oracle_df['Color Category'].str.contains('Blue', na=False)) & 
    (oracle_df['Type'].str.contains('Creature', na=False)) &
    (oracle_df['Oracle Text'].str.contains('draw.*card|when.*enters.*draw', case=False, na=False))
]

print("CREATURES THAT DRAW CARDS:")
for _, creature in card_draw_creatures.head(10).iterrows():
    oracle_text = creature['Oracle Text'] if pd.notna(creature['Oracle Text']) else 'No text'
    print(f"\n{creature['name']} ({creature['CMC']} CMC) - {creature['Type']}")
    print(f"  Text: {oracle_text[:100]}...")

# Look for pure card draw spells
pure_draw_spells = oracle_df[
    (oracle_df['Color Category'].str.contains('Blue', na=False)) & 
    (~oracle_df['Type'].str.contains('Creature', na=False)) &
    (oracle_df['Oracle Text'].str.contains('draw.*card|scry|surveil', case=False, na=False)) &
    (oracle_df['CMC'] <= 3)  # Focus on efficient spells
]

print(f"\n{'='*50}")
print("EFFICIENT CARD DRAW SPELLS (3 CMC or less):")
for _, spell in pure_draw_spells.head(15).iterrows():
    oracle_text = spell['Oracle Text'] if pd.notna(spell['Oracle Text']) else 'No text'
    print(f"\n{spell['name']} ({spell['CMC']} CMC) - {spell['Type']}")
    print(f"  Text: {oracle_text[:120]}...")

# Check for divination effects
divination_effects = oracle_df[
    (oracle_df['Color Category'].str.contains('Blue', na=False)) & 
    (oracle_df['Oracle Text'].str.contains('draw.*two|draw.*three', case=False, na=False))
]

print(f"\n{'='*50}")
print("CARD ADVANTAGE SPELLS (Draw 2+ cards):")
for _, spell in divination_effects.head(8).iterrows():
    oracle_text = spell['Oracle Text'] if pd.notna(spell['Oracle Text']) else 'No text'
    print(f"\n{spell['name']} ({spell['CMC']} CMC) - {spell['Type']}")
    print(f"  Text: {oracle_text[:120]}...")

SEARCHING FOR BETTER ALTERNATIVES:
CREATURES THAT DRAW CARDS:

Looter il-Kor (2 CMC) - Creature - Kor Rogue
  Text: Shadow (This creature can block or be blocked by only creatures with shadow.) | Whenever this creatu...

Merfolk Looter (2 CMC) - Creature - Merfolk Rogue
  Text: {T}: Draw a card, then discard a card....

Moon-Circuit Hacker (2 CMC) - Enchantment Creature - Human Ninja
  Text: Ninjutsu {U} ({U}, Return an unblocked attacker you control to hand: Put this card onto the battlefi...

Cloudkin Seer (3 CMC) - Creature - Elemental Wizard
  Text: Flying | When this creature enters, draw a card....

Jhessian Thief (3 CMC) - Creature - Human Rogue
  Text: Prowess (Whenever you cast a noncreature spell, this creature gets +1/+1 until end of turn.) | Whene...

Serum Visionary (3 CMC) - Creature - Vedalken Wizard
  Text: When this creature enters, draw a card, then scry 2....

Vizier of Tumbling Sands (3 CMC) - Creature - Human Cleric
  Text: {T}: Untap another target permanent. | Cy

In [13]:
# FINAL ANALYSIS AND RECOMMENDATIONS
print("🎯 FINAL ANALYSIS: Blue Card Draw Deck")
print("=" * 60)

print("CURRENT DECK COHERENCE SCORE: 1.98 (needs improvement)")

print("\n📊 CURRENT DECK COMPOSITION:")
print("✅ GOOD FITS (keep these):")
print("  - Opt (perfect 1-mana cantrip)")
print("  - Preordain (excellent card selection)")  
print("  - Brainstorm (classic card manipulation)")
print("  - Serum Visionary (creature that draws)")
print("  - Faerie Seer (creature that scrys)")

print("\n❌ POOR FITS (should replace):")
print("  - Repeal & Into the Roil (bounce, not draw focused)")
print("  - Young Blue Dragon (big creature, wrong theme)")
print("  - Candy Trail (artifact, doesn't fit)")
print("  - Condescend (counterspell, not draw)")

print("\n🔄 SUGGESTED REPLACEMENTS:")
print("Replace Repeal → Serum Visions (pure card draw + scry)")
print("Replace Into the Roil → Consider (modern card selection)")
print("Replace Young Blue Dragon → Mulldrifter (card advantage creature)")
print("Replace Candy Trail → Quick Study (instant-speed card draw)")
print("Replace Condescend → Gitaxian Probe (free card draw)")

print("\n🎨 IMPROVED THEME DEFINITION:")
print("Current keywords:", theme_keywords['Card Draw'])
print("Suggested additions:")
theme_keywords_improved = theme_keywords['Card Draw'] + ['surveil', 'look at', 'reveal', 'impulse draw', 'foretell']
print("Enhanced keywords:", theme_keywords_improved)

print("\n📋 IDEAL BLUE CARD DRAW DECK COMPOSITION:")
print("1. Opt (1 CMC cantrip)")
print("2. Preordain (1 CMC selection)")
print("3. Brainstorm (1 CMC manipulation)")  
print("4. Serum Visions (1 CMC draw + scry)")
print("5. Consider (1 CMC surveil + draw)")
print("6. Gitaxian Probe (0 CMC peek + draw)")
print("7. Quick Study (3 CMC instant draw 2)")
print("8. Serum Visionary (3 CMC creature + draw)")
print("9. Faerie Seer (1 CMC creature + scry)")
print("10. Cloudkin Seer (3 CMC flyer + draw)")
print("11. Mulldrifter (5 CMC big draw)")
print("12. Behold the Multiverse (4 CMC foretell draw)")
print("13. Lórien Revealed (5 CMC big draw)")

print("\n✨ EXPECTED IMPROVEMENTS:")
print("- Higher coherence score (target: 2.5+)")
print("- More consistent card selection theme")
print("- Better mana curve for card draw")
print("- Pure focus on draw/selection effects")

🎯 FINAL ANALYSIS: Blue Card Draw Deck
CURRENT DECK COHERENCE SCORE: 1.98 (needs improvement)

📊 CURRENT DECK COMPOSITION:
✅ GOOD FITS (keep these):
  - Opt (perfect 1-mana cantrip)
  - Preordain (excellent card selection)
  - Brainstorm (classic card manipulation)
  - Serum Visionary (creature that draws)
  - Faerie Seer (creature that scrys)

❌ POOR FITS (should replace):
  - Repeal & Into the Roil (bounce, not draw focused)
  - Young Blue Dragon (big creature, wrong theme)
  - Candy Trail (artifact, doesn't fit)
  - Condescend (counterspell, not draw)

🔄 SUGGESTED REPLACEMENTS:
Replace Repeal → Serum Visions (pure card draw + scry)
Replace Into the Roil → Consider (modern card selection)
Replace Young Blue Dragon → Mulldrifter (card advantage creature)
Replace Candy Trail → Quick Study (instant-speed card draw)
Replace Condescend → Gitaxian Probe (free card draw)

🎨 IMPROVED THEME DEFINITION:
Current keywords: ['draw', 'card', 'hand', 'library', 'cantrip', 'scry']
Suggested additions

In [14]:
# Let's reload the consts to get the updated keywords and analyze each one
import importlib
import src.consts
importlib.reload(src.consts)
from src.consts import theme_keywords

print("🔍 ANALYZING CURRENT CARD DRAW KEYWORDS FOR REMOVAL")
print("=" * 60)

current_keywords = theme_keywords['Card Draw']
print("Current Card Draw keywords:", current_keywords)

print(f"\n{'='*50}")
print("KEYWORD EFFECTIVENESS ANALYSIS:")
print("=" * 50)

# Test each keyword against the oracle_df to see how many relevant cards it matches
keyword_analysis = {}

for keyword in current_keywords:
    # Count blue cards that contain this keyword
    if '.*' in keyword:  # Handle regex patterns
        import re
        pattern = keyword
        blue_matches = oracle_df[
            (oracle_df['Color Category'].str.contains('Blue', na=False)) & 
            (oracle_df['Oracle Text'].str.contains(pattern, case=False, na=False, regex=True))
        ]
    else:
        blue_matches = oracle_df[
            (oracle_df['Color Category'].str.contains('Blue', na=False)) & 
            (oracle_df['Oracle Text'].str.contains(keyword, case=False, na=False))
        ]
    
    keyword_analysis[keyword] = {
        'count': len(blue_matches),
        'cards': blue_matches['name'].tolist()[:5]  # Show first 5 examples
    }
    
    print(f"\n'{keyword}': {len(blue_matches)} blue cards")
    if len(blue_matches) > 0:
        print(f"  Examples: {', '.join(blue_matches['name'].tolist()[:3])}")
    else:
        print("  ⚠️  NO MATCHES - Consider removing")

print(f"\n{'='*50}")
print("KEYWORD QUALITY ASSESSMENT:")
print("=" * 50)

print("HIGH VALUE KEYWORDS (keep these):")
for keyword, data in keyword_analysis.items():
    if data['count'] >= 5:  # Good number of matches
        print(f"  ✅ '{keyword}' - {data['count']} matches")

print("\nMODERATE VALUE KEYWORDS (review these):")
for keyword, data in keyword_analysis.items():
    if 1 <= data['count'] < 5:
        print(f"  ⚠️  '{keyword}' - {data['count']} matches")

print("\nLOW/NO VALUE KEYWORDS (consider removing):")
for keyword, data in keyword_analysis.items():
    if data['count'] == 0:
        print(f"  ❌ '{keyword}' - {data['count']} matches")

print(f"\n{'='*50}")
print("SPECIFIC KEYWORD RECOMMENDATIONS:")
print("=" * 50)

🔍 ANALYZING CURRENT CARD DRAW KEYWORDS FOR REMOVAL
Current Card Draw keywords: ['draw', 'card', 'hand', 'library', 'cantrip', 'scry', 'surveil', 'look at', 'reveal', 'impulse draw', 'foretell', 'exile.*play']

KEYWORD EFFECTIVENESS ANALYSIS:

'draw': 27 blue cards
  Examples: Looter il-Kor, Merfolk Looter, Moon-Circuit Hacker

'card': 42 blue cards
  Examples: Faerie Seer, Deranged Assistant, Looter il-Kor

'hand': 15 blue cards
  Examples: Moon-Circuit Hacker, Man-o'-War, Winter Eladrin

'library': 10 blue cards
  Examples: Faerie Seer, Deranged Assistant, Cruel Witness

'cantrip': 0 blue cards
  ⚠️  NO MATCHES - Consider removing

'scry': 8 blue cards
  Examples: Faerie Seer, Serum Visionary, Young Blue Dragon

'surveil': 2 blue cards
  Examples: Cruel Witness, Consider

'look at': 7 blue cards
  Examples: Faerie Seer, Cruel Witness, Condescend

'reveal': 1 blue cards
  Examples: Lórien Revealed

'impulse draw': 0 blue cards
  ⚠️  NO MATCHES - Consider removing

'foretell': 1 blue ca

In [15]:
print("SPECIFIC REMOVAL RECOMMENDATIONS:")
print("=" * 40)

print("❌ REMOVE THESE KEYWORDS:")
print("  - 'cantrip' - Not used in oracle text (it's slang/community term)")
print("  - 'impulse draw' - Too specific phrase, not in oracle text")
print("  - 'exile.*play' - No matches, regex may be too complex")

print("\n🤔 KEYWORDS TO CONSIDER REMOVING:")
print("  - 'reveal' - Only 1 match, very specific")
print("  - 'foretell' - Only 1 match, mechanic-specific")

print("\n✅ KEYWORDS TO DEFINITELY KEEP:")
print("  - 'draw' - Core keyword (27 matches)")
print("  - 'card' - Essential term (42 matches)")
print("  - 'hand' - Common reference (15 matches)")
print("  - 'library' - Important for card selection (10 matches)")
print("  - 'scry' - Key mechanic (8 matches)")
print("  - 'look at' - Card selection phrase (7 matches)")

print("\n🔄 POTENTIAL ADDITIONS TO CONSIDER:")
print("  - 'cycling' - Draw replacement effect")
print("  - 'search' - For tutoring effects")
print("  - 'target player draws' - For divination effects")

# Let's test these potential additions
print(f"\n{'='*40}")
print("TESTING POTENTIAL NEW KEYWORDS:")
print("=" * 40)

potential_keywords = ['cycling', 'search', 'target player draws', 'draws.*card']

for keyword in potential_keywords:
    if keyword == 'draws.*card':  # regex
        matches = oracle_df[
            (oracle_df['Color Category'].str.contains('Blue', na=False)) & 
            (oracle_df['Oracle Text'].str.contains(keyword, case=False, na=False, regex=True))
        ]
    else:
        matches = oracle_df[
            (oracle_df['Color Category'].str.contains('Blue', na=False)) & 
            (oracle_df['Oracle Text'].str.contains(keyword, case=False, na=False))
        ]
    
    print(f"\n'{keyword}': {len(matches)} blue cards")
    if len(matches) > 0:
        print(f"  Examples: {', '.join(matches['name'].tolist()[:3])}")

print(f"\n{'='*50}")
print("🎯 FINAL RECOMMENDATIONS:")
print("=" * 50)
print("REMOVE: 'cantrip', 'impulse draw', 'exile.*play'")
print("KEEP: 'draw', 'card', 'hand', 'library', 'scry', 'look at', 'surveil'") 
print("CONSIDER REMOVING: 'reveal', 'foretell' (too specific)")
print("CONSIDER ADDING: 'cycling', 'draws.*card' (if regex works)")

SPECIFIC REMOVAL RECOMMENDATIONS:
❌ REMOVE THESE KEYWORDS:
  - 'cantrip' - Not used in oracle text (it's slang/community term)
  - 'impulse draw' - Too specific phrase, not in oracle text
  - 'exile.*play' - No matches, regex may be too complex

🤔 KEYWORDS TO CONSIDER REMOVING:
  - 'reveal' - Only 1 match, very specific
  - 'foretell' - Only 1 match, mechanic-specific

✅ KEYWORDS TO DEFINITELY KEEP:
  - 'draw' - Core keyword (27 matches)
  - 'card' - Essential term (42 matches)
  - 'hand' - Common reference (15 matches)
  - 'library' - Important for card selection (10 matches)
  - 'scry' - Key mechanic (8 matches)
  - 'look at' - Card selection phrase (7 matches)

🔄 POTENTIAL ADDITIONS TO CONSIDER:
  - 'cycling' - Draw replacement effect
  - 'search' - For tutoring effects
  - 'target player draws' - For divination effects

TESTING POTENTIAL NEW KEYWORDS:

'cycling': 4 blue cards
  Examples: Vizier of Tumbling Sands, Striped Riverwinder, Miscalculation

'search': 1 blue cards
  Example

In [18]:
# Let's test optimization specifically for Blue Card Draw deck
print("🔧 TESTING BLUE CARD DRAW OPTIMIZATION")
print("=" * 60)

# First, let's check the current coherence with updated keywords
import importlib
import src.coherence
importlib.reload(src.coherence)
from src.coherence import analyze_deck_theme_coherence_enhanced

# Reload consts to get updated keywords
import src.consts
importlib.reload(src.consts)

# Get current Blue Card Draw coherence
current_coherence = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
blue_coherence = current_coherence.get('Blue Card Draw', {})

print("CURRENT STATE:")
print(f"Blue Card Draw coherence: {blue_coherence.get('overall_coherence', 'N/A')}")

# Check what cards are currently in the deck
blue_deck = cube_df[cube_df['Tags'] == 'Blue Card Draw']
print(f"\nCurrent Blue Card Draw deck ({len(blue_deck)} cards):")
for _, card in blue_deck.iterrows():
    print(f"  - {card['Name']}")

# Now let's see what better alternatives exist in the oracle
print(f"\n{'='*50}")
print("TESTING FOR POTENTIAL SWAPS:")

# Look for high-quality blue card draw cards not in the current deck
current_card_names = set(blue_deck['Name'].tolist())

# Find blue cards that match our improved keywords
from src.consts import theme_keywords
card_draw_keywords = theme_keywords['Card Draw']

print(f"Using keywords: {card_draw_keywords}")

# Find potential replacements
good_blue_cards = oracle_df[
    (oracle_df['Color Category'].str.contains('Blue', na=False)) & 
    (oracle_df['Oracle Text'].str.contains('|'.join(card_draw_keywords), case=False, na=False))
]

# Filter out cards already in deck
potential_swaps = good_blue_cards[~good_blue_cards['name'].isin(current_card_names)]

print(f"\nFound {len(potential_swaps)} potential blue card draw cards not in current deck:")
for _, card in potential_swaps.head(10).iterrows():
    oracle_text = card['Oracle Text'] if pd.notna(card['Oracle Text']) else 'No text'
    print(f"  + {card['name']} ({card['CMC']} CMC): {oracle_text[:60]}...")

# Test if optimization function can find these
print(f"\n{'='*50}")
print("TESTING OPTIMIZATION FUNCTION:")

import src.process
importlib.reload(src.process)
from src.process import optimize_deck_coherence, clear_swap_history

# Clear swap history
clear_swap_history()

print("Running optimize_deck_coherence...")
old_cube_df = cube_df.copy()
new_cube_df = optimize_deck_coherence(cube_df=cube_df, oracle_df=oracle_df)

# Check if anything changed
if new_cube_df.equals(old_cube_df):
    print("❌ No changes made by optimization")
else:
    print("✅ Optimization made changes!")
    
    # Show what changed in Blue Card Draw deck
    old_blue = old_cube_df[old_cube_df['Tags'] == 'Blue Card Draw']['Name'].tolist()
    new_blue = new_cube_df[new_cube_df['Tags'] == 'Blue Card Draw']['Name'].tolist()
    
    removed = set(old_blue) - set(new_blue)
    added = set(new_blue) - set(old_blue)
    
    if removed:
        print(f"Removed: {', '.join(removed)}")
    if added:
        print(f"Added: {', '.join(added)}")
        
    # Check new coherence
    new_coherence = analyze_deck_theme_coherence_enhanced(new_cube_df, oracle_df)
    new_blue_coherence = new_coherence.get('Blue Card Draw', {}).get('overall_coherence', 'N/A')
    print(f"New Blue Card Draw coherence: {new_blue_coherence}")
    print(f"Improvement: {new_blue_coherence - blue_coherence.get('overall_coherence', 0):.3f}")

🔧 TESTING BLUE CARD DRAW OPTIMIZATION
CURRENT STATE:
Blue Card Draw coherence: 1.9830769230769232

Current Blue Card Draw deck (13 cards):
  - Behold the Multiverse
  - Lórien Revealed
  - Moon-Circuit Hacker
  - Opt
  - Young Blue Dragon
  - Brainstorm
  - Condescend
  - Preordain
  - Repeal
  - Into the Roil
  - Serum Visionary
  - Faerie Seer
  - Candy Trail

TESTING FOR POTENTIAL SWAPS:
Using keywords: ['draw', 'card', 'hand', 'library', 'scry', 'surveil', 'look at', 'cycling']

Found 34 potential blue card draw cards not in current deck:
  + Deranged Assistant (2 CMC): {T}, Mill a card: Add {C}. (To mill a card, put the top card...
  + Looter il-Kor (2 CMC): Shadow (This creature can block or be blocked by only creatu...
  + Merfolk Looter (2 CMC): {T}: Draw a card, then discard a card....
  + Cloudkin Seer (3 CMC): Flying | When this creature enters, draw a card....
  + Eldrazi Skyspawner (3 CMC): Devoid (This card has no color.) | Flying | When this creatu...
  + Jhessian Thief 

Analyzing deck: Blue Card Draw

Current coherence: 2.0

Expected themes: Card Draw

Deck colors: U

Found 39 candidate cards to consider

Identified 13 cards as potential removal candidates

# 🔄 Swap Recommendations for Blue Card Draw

❌ **No beneficial swaps found.** The deck may already be well-optimized, or there may not be suitable replacement cards available.

No available swaps to apply for this deck.
Card: Behold the Multiverse, Score: 4, Themes: ['Card Draw']
Card: Lórien Revealed, Score: 4, Themes: ['Card Draw']
Card: Moon-Circuit Hacker, Score: 3, Themes: ['Card Draw']
Card: Opt, Score: 4, Themes: ['Card Draw']
Card: Young Blue Dragon, Score: 3, Themes: ['Card Draw']
Card: Brainstorm, Score: 4, Themes: ['Card Draw']
Card: Condescend, Score: 3, Themes: ['Card Draw']
Card: Preordain, Score: 4, Themes: ['Card Draw']
Card: Repeal, Score: 3, Themes: ['Card Draw']
Card: Into the Roil, Score: 3, Themes: ['Card Draw']
Card: Serum Visionary, Score: 3, Themes: ['Card Draw']
Card: Faerie Seer, Score: 3, Themes: ['Card Draw']
Card: Candy Trail, Score: 3, Themes: ['Card Draw']
1.9830769230769232
{'expected_themes': ['Card Draw'], 'theme_score': 3.3846153846153846, 'theme_matches': [{'card': 'Behold the Multiverse', 'score': 4, 'themes': ['Card Draw']}, {'card': 'Lórien Revealed', 'score': 4, 'themes': ['Card Draw']}, {'card': 'Moon-Circuit Hacker', 

In [20]:
# Let's investigate why no beneficial swaps are found
print("🔍 INVESTIGATING WHY NO BENEFICIAL SWAPS FOUND")
print("=" * 60)

# First, let's check the theme criteria for Card Draw
print("CHECKING THEME CRITERIA:")
print("=" * 30)

card_draw_criteria = theme_criteria.get('Card Draw', {})
print(f"Card Draw theme criteria: {card_draw_criteria}")

if not card_draw_criteria:
    print("❌ NO THEME CRITERIA DEFINED FOR 'Card Draw'!")
    print("This is likely why optimization isn't working properly.")
    print("\nThe available theme criteria are:")
    for theme_name in theme_criteria.keys():
        print(f"  - {theme_name}")
    
    print("\n💡 SOLUTION: The theme criteria only contains specific theme names,")
    print("but our deck is named 'Blue Card Draw' which doesn't match any criteria.")
    print("The optimization likely isn't recognizing this as a valid theme.")

print(f"\n{'='*50}")
print("CHECKING DECK THEME EXTRACTION:")

# Let's see what theme the extraction function returns for "Blue Card Draw"
from src.deck import extract_theme_from_deck_name

extracted_theme = extract_theme_from_deck_name("Blue Card Draw")
print(f"Extracted theme from 'Blue Card Draw': {extracted_theme}")

# Check all deck names and their extracted themes
print(f"\nAll deck themes in cube:")
unique_decks = cube_df['Tags'].unique()
for deck_name in unique_decks:
    extracted = extract_theme_from_deck_name(deck_name)
    has_criteria = extracted in theme_criteria if extracted else False
    print(f"  {deck_name} → {extracted} (criteria: {'✅' if has_criteria else '❌'})")

print(f"\n{'='*50}")
print("ROOT CAUSE ANALYSIS:")
print("The optimization is probably failing because:")
print("1. 'Blue Card Draw' → extracts to 'Card Draw'")
print("2. 'Card Draw' is not in theme_criteria")
print("3. Without criteria, the coherence scoring doesn't work properly")
print("4. Without proper scoring, no swaps appear beneficial")

print(f"\n💡 SOLUTION:")
print("We need to either:")
print("1. Add 'Card Draw' to theme_criteria in consts.py, or") 
print("2. Rename the deck to match an existing theme, or")
print("3. Fix the theme extraction to map to an existing theme")

🔍 INVESTIGATING WHY NO BENEFICIAL SWAPS FOUND
CHECKING THEME CRITERIA:
Card Draw theme criteria: {}
❌ NO THEME CRITERIA DEFINED FOR 'Card Draw'!
This is likely why optimization isn't working properly.

The available theme criteria are:
  - Graveyard
  - Aggro
  - Beatdown
  - Control
  - Flying
  - Big Creatures
  - Stompy
  - Tokens
  - Artifacts
  - Red Artifacts
  - Sacrifice
  - Equipment
  - Small Creatures

💡 SOLUTION: The theme criteria only contains specific theme names,
but our deck is named 'Blue Card Draw' which doesn't match any criteria.
The optimization likely isn't recognizing this as a valid theme.

CHECKING DECK THEME EXTRACTION:
Extracted theme from 'Blue Card Draw': ['Card Draw']

All deck themes in cube:


TypeError: unhashable type: 'list'

In [21]:
# Now let's test optimization after adding Card Draw theme criteria
print("🔧 TESTING OPTIMIZATION AFTER ADDING CARD DRAW CRITERIA")
print("=" * 60)

# Reload consts to get the new theme criteria
import importlib
import src.consts
importlib.reload(src.consts)
from src.consts import theme_criteria, theme_keywords

# Check if Card Draw is now in theme criteria
card_draw_criteria = theme_criteria.get('Card Draw', {})
print(f"Card Draw theme criteria: {card_draw_criteria}")

if card_draw_criteria:
    print("✅ Card Draw theme criteria is now defined!")
    
    # Reload the optimization function to pick up new criteria
    import src.process
    importlib.reload(src.process)
    from src.process import optimize_deck_coherence, clear_swap_history
    
    # Clear swap history
    clear_swap_history()
    
    # Test optimization again
    print(f"\n{'='*50}")
    print("RUNNING OPTIMIZATION WITH NEW CRITERIA:")
    
    # Get current state
    old_cube_df = cube_df.copy()
    old_blue_deck = old_cube_df[old_cube_df['Tags'] == 'Blue Card Draw']
    print(f"Original deck:")
    for _, card in old_blue_deck.iterrows():
        print(f"  - {card['Name']}")
    
    # Get current coherence
    import src.coherence
    importlib.reload(src.coherence)
    from src.coherence import analyze_deck_theme_coherence_enhanced
    
    old_coherence = analyze_deck_theme_coherence_enhanced(old_cube_df, oracle_df)
    old_blue_coherence = old_coherence.get('Blue Card Draw', {}).get('overall_coherence', 0)
    print(f"\nOriginal coherence: {old_blue_coherence:.3f}")
    
    # Run optimization
    print("\nRunning optimization...")
    new_cube_df = optimize_deck_coherence(cube_df=old_cube_df, oracle_df=oracle_df)
    
    # Check results
    if new_cube_df.equals(old_cube_df):
        print("❌ Still no changes made by optimization")
        print("This might mean the current deck is already optimal,")
        print("or there are other issues with the optimization logic.")
    else:
        print("✅ Optimization made changes!")
        
        # Show what changed
        new_blue_deck = new_cube_df[new_cube_df['Tags'] == 'Blue Card Draw']
        
        old_cards = set(old_blue_deck['Name'].tolist())
        new_cards = set(new_blue_deck['Name'].tolist())
        
        removed = old_cards - new_cards
        added = new_cards - old_cards
        
        if removed:
            print(f"\nRemoved cards: {', '.join(removed)}")
        if added:
            print(f"Added cards: {', '.join(added)}")
        
        # Check new coherence
        new_coherence = analyze_deck_theme_coherence_enhanced(new_cube_df, oracle_df)
        new_blue_coherence = new_coherence.get('Blue Card Draw', {}).get('overall_coherence', 0)
        
        print(f"\nNew coherence: {new_blue_coherence:.3f}")
        print(f"Improvement: {new_blue_coherence - old_blue_coherence:.3f}")
        
        print(f"\nFinal deck:")
        for _, card in new_blue_deck.iterrows():
            print(f"  - {card['Name']}")
            
else:
    print("❌ Card Draw theme criteria still not found")

🔧 TESTING OPTIMIZATION AFTER ADDING CARD DRAW CRITERIA
Card Draw theme criteria: {'keywords': ['draw', 'card', 'hand', 'library', 'scry', 'surveil', 'look at', 'cycling'], 'abilities': ['enters', 'when', 'draw', 'scry', 'look', 'search'], 'stats_matter': False, 'utility_bonus': 2.5, 'evasion_bonus': 1.0, 'instant_speed_bonus': 1.5, 'card_advantage_bonus': 3.0}
✅ Card Draw theme criteria is now defined!
Swap history cleared

RUNNING OPTIMIZATION WITH NEW CRITERIA:
Original deck:
  - Behold the Multiverse
  - Lórien Revealed
  - Moon-Circuit Hacker
  - Opt
  - Young Blue Dragon
  - Brainstorm
  - Condescend
  - Preordain
  - Repeal
  - Into the Roil
  - Serum Visionary
  - Faerie Seer
  - Candy Trail

Original coherence: 11.983

Running optimization...


Analyzing deck: Blue Tempo

Current coherence: 2.1

Expected themes: Tempo

Deck colors: U

...

- **Lose Focus** (Theme Score: 2.0, CMC: 2.0)

### Cards to Add:

- **Moon-Circuit Hacker** (Theme Score: 5.0) - from Blue Card Draw

- **Bubble Snare** (Theme Score: 5.0) - from Azorius Evasion/Flying

2.3261538461538462
{'expected_themes': ['Tempo'], 'theme_score': 4.076923076923077, 'theme_matches': [{'card': 'Peregrine Drake', 'score': 5, 'themes': ['Tempo']}, {'card': 'Miscalculation', 'score': 3, 'themes': ['Tempo']}, {'card': 'Moon-Circuit Hacker', 'score': 5, 'themes': ['Tempo']}, {'card': 'Vizier of Tumbling Sands', 'score': 5, 'themes': ['Tempo']}, {'card': 'Cloudkin Seer', 'score': 4, 'themes': ['Tempo']}, {'card': 'Mulldrifter', 'score': 4, 'themes': ['Tempo']}, {'card': 'Phantom Interference', 'score': 2, 'themes': ['Tempo']}, {'card': 'Zephyr Winder', 'score': 4, 'themes': ['Tempo']}, {'card': 'Frost Trickster', 'score': 5, 'themes': ['Tempo']}, {'card': 'Bubble Snare', 'score': 5, 'themes': ['Tempo']}, {'card': 'Cruel Witness', 'score': 2, 'themes': ['Tempo']}, {'card': 'Pestermite', 'score': 6, 'themes': ['Tempo']}, {'card': 'Shipwreck Dowser', 'score': 3, 'themes': ['Tempo']}], 'color_coherence': 1.0, 'color_issues': [], 'mana_curve_score': 0.9384615384615386, 'mana_c